In [ ]:
# grabbing src code
import sys
sys.path.insert(0, "../src")

# usual suspects
import numpy as np
import matplotlib.pyplot as plt

import scipy.io.wavfile as wav
from scipy.fftpack import fft, ifft
from scipy.signal import fftconvolve, convolve
from scipy.linalg import toeplitz, lstsq
from scipy.signal import correlate, convolve

from IPython.display import Audio, display

# our stuff
import Zero_pad as zp
from mask_maker import create_n_channel_data, mel_stft, random_STAT
import Conj_grad as cg
from rir_estim import RoomResponseEstimator, record_response, calculate_rir

# misc
import sounddevice as sd
from tqdm import tqdm
import pickle as pkl

%matplotlib inline

sd.default.device = 1
print (sd.query_devices())

In [ ]:
#create sine sweep and estimator
fs = 44100
estimator = RoomResponseEstimator(duration=2.0, low=30.0, high=15000.0, Fs=fs)
sine_sweep = estimator.probe()

In [ ]:
#record all responses
n_speakers = 6
n_mic = 2
responses = [[[] for i in range(n_speakers)] for j in range(n_mic)]
for i in range(n_speakers):
    resp  = record_response(sine_sweep, i , fs, fs, n_mic, verbose = False)
    resp_array = np.asarray(resp)
    for j in range(n_mic):
        responses[j][i].append(list(resp_array[:,j]))

In [ ]:
#calculate RIRs
RIRS = [[[] for i in range(n_speakers)] for j in range(n_mic)]
for i in range(n_speakers):
    for j in range(n_mic):
        rir = calculate_rir(responses[j][i][0], 1024, estimator, shorten = 25000, verbose = False)
        RIRS[j][i].append(rir)
        
for i in range(n_speakers):
    for j in range(n_mic):
        RIRS[j][i] = RIRS[j][i][0]

In [ ]:
# Start of the jointly optimization process
# Variables we can play with
M = 100000 # filter length
delay = 10000 # delay of the signals in number of samples
n_speaker = 6

# Read in the original sound signals
n_signals = 2
fs = np.empty((n_signals,), dtype=object)
orig_sound = np.empty((n_signals,), dtype=object)
[fs[0], orig_sound[0]] = wav.read('../sounds/news.wav')
[fs[1], orig_sound[1]] = wav.read('../sounds/metalking_44k.wav')


#This is for clipping to 4 sec
orig_sound[0] = 2.3/3.*orig_sound[0][5*fs[0]:5*fs[0] + 7*fs[0]]
orig_sound[1] = orig_sound[1][:7*fs[1]] * 0

# Zero pad the shorter signals so that each signal has the same length
# Also, at the same time obtain the delayed version of the signal
length = []
for i in range(n_signals):
    length.append(len(orig_sound[i]))

sig_length = max(length)

delay_sound = np.empty((n_signals,), dtype=object)
for i in range(n_signals):
    delay_sound[i] = zp.front_pad(
        zp.back_pad(orig_sound[i], (sig_length - len(orig_sound[i]))), delay).astype(np.int16)
    orig_sound[i] = zp.back_pad(orig_sound[i] , (sig_length - len(orig_sound[i]) + delay)).astype(np.int16)

mean = 0
std = 10 
num_samples = len(orig_sound[0])
orig_sound[0] = 250 * np.random.normal(mean, std, size=N)
orig_sound[1] = 250 * np.random.normal(mean, std, size=N)

In [ ]:
Audio(orig_sound[0], rate = fs[0])

In [ ]:
Audio(orig_sound[1], rate = fs[1])

In [ ]:
# Zero pad the RIRs so that they have the same length
rir_length = []
for i in range(n_mic):
    for j in range(n_speaker):
        rir_length.append(len(RIRS[i][j]))
K = max(rir_length)

for i in range(n_mic):
    for j in range(n_speaker):
        RIRS[i][j] = zp.back_pad(RIRS[i][j] , K - len(RIRS[i][j]))

# Parameters for chopping
num_samples_shift = 100
samples_per_chunk = 500
speaker_offset = 0
        
# Obtain the chopped signals and the masks
speaker_data = np.empty((n_signals,), dtype=object)
mask = np.empty((n_signals,), dtype=object)
for i in range(n_signals):
    speaker_data[i], mask[i] = np.asarray(
        create_n_channel_data(num_samples_shift, samples_per_chunk , speaker_offset, orig_sound[i], n_speaker, random_cycling = True))

# Form the target b vector and pass the parameters into the CG method to calculate for the filter g
b = []
for i in range(n_signals):
    b.append(zp.back_pad(list(delay_sound[i]), M+N+K - 2 - len(delay_sound[i]))) # Matching up the dimension
b_vec = np.array(b).flatten()  

In [ ]:
iter_num = 100 # number of iterations for our CG
g_hat = cg.create_ghat(speaker_data, b_vec, iter_num, n_signals, RIRS, M , init_guess = 'uni_random', plot=True)

In [ ]:
# Getting the filter g_hat for each speaker / each focusing zone
g_hat_len = len(g_hat)
g_hat_speaker = []
for i in range(n_speakers*n_signals):
    g_hat_speaker.append(g_hat[int(i*(g_hat_len/n_signals/n_speakers)):int((i+1)*(g_hat_len/n_signals/n_speakers))])

In [ ]:
y_play = []
count = 0
for i in range(n_speakers):
    y_play_sum = np.zeros((len(speaker_data[0][:, 0]) + len(g_hat_speaker[count]) - 1,))
    
    for j in range(n_signals):
        y_play_sum += convolve( speaker_data[j][:, i], g_hat_speaker[count])
        count += 1
        
    y_play.append(y_play_sum)
    
    print(y_play[i].shape)
    plt.figure()
    plt.title('Signal chunk * filter')
    plt.plot(y_play[i])
    plt.show()
    
y_play = np.asarray(y_play)
print(y_play.shape)

In [ ]:
final_signal = np.zeros( (len(y_play[0]), n_speakers))
for i in range(n_speakers):
    final_signal[:,i] = y_play[i]

# pad so that soundevice will keep recording and we can grab the last echoes
padded_final_signal = np.vstack( ( final_signal, np.zeros( ( 44100 , n_speakers ) ) ) )
padded_final_signal *= .0000008 # arbitrary scaling factor


In [ ]:
#This actually plays and records
num_mics_total = 5
y_mic = sd.playrec( padded_final_signal , fs[0], channels=num_mics_total )
sd.wait()

for i in range(num_mics_total):
    display(Audio( y_mic[:,i], rate = fs[0]))